In [249]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr

In [227]:
Predicted = pd.read_csv("./Seurat/Outputs/output_exp0.csv")
Target=pd.read_csv("./Data/exp0/target.csv")

In [228]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
Predicted.drop(columns="prediction.score.max", inplace=True)


In [229]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [230]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [231]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [232]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [233]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [234]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [235]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [236]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [237]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

/opt/homebrew/Caskroom/miniforge/base/envs/ML/lib/python3.10/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/opt/homebrew/Caskroom/miniforge/base/envs/ML/lib/python3.10/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [241]:
np.mean(np.nan_to_num(d))

0.3038751748184647

In [242]:
d

array([ 0.05241387,  0.54008183,  0.57325816,  0.02955036, -0.012541  ,
        0.54524885,  0.37554739,  0.14581198,  0.30864388,  0.54340623,
        0.5616401 ,  0.87269385,         nan,         nan, -0.04801997,
        0.69900197,  0.51107057,  0.16241332, -0.03695507,  0.10481629,
        0.01846964,  0.76096561,  0.65865811,  0.18057726,  0.82959974,
        0.44262121,  0.26088289,  0.02507962, -0.01362955,  0.11509357,
               nan,  0.57841163, -0.03640285, -0.00407546,  0.23646665,
        0.67283644,  0.78078228,  0.81824348,  0.24773536,  0.75932969,
               nan,         nan,  0.25498078,  0.49241535, -0.02394035,
        0.60577396,  0.37484487,         nan, -0.07391898])

## Jenson Shannon Divergence

In [244]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.623981827553795

## F1 Score, Precision, Recall

In [246]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [247]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [248]:
d

,Precision,Recall,F1-score
0.100,0.328802,0.348129,0.338190
0.200,0.229634,0.515232,0.317681
0.300,0.116016,0.662757,0.197466
0.010,0.475286,0.269259,0.343767
0.001,0.473346,0.298603,0.366196
